<a href="https://colab.research.google.com/github/leonardojfrenner/llm_hugging_face_langChain/blob/main/models_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers einops accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.3 MB/s eta 0:00:00


In [2]:
!pip install -q langchain
!pip install -q langchain-community
!pip install -q langchain-huggingface
!pip install -q langchainhub
!pip install -q langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.9 MB/s eta 0:00:0

In [3]:
import torch
import os
import getpass

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [4]:
from langchain_huggingface import ChatHuggingFace
from langchain_core.messages import HumanMessage


In [5]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


In [6]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [7]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [8]:
pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [9]:
from langchain_huggingface import ChatHuggingFace
from langchain_core.messages import HumanMessage


In [10]:
msgs = [
    SystemMessage(content = "Você é um assistente e está respondendo perguntas gerais."),
    HumanMessage(content = "Explique para mim brevemente o conceito de IA.")
]

In [11]:
chat_model = ChatHuggingFace(llm = llm)

In [12]:
model_template = tokenizer.chat_template
model_template

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

In [13]:
chat_model._to_chat_prompt(msgs)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nVocê é um assistente e está respondendo perguntas gerais.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplique para mim brevemente o conceito de IA.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [14]:
res = chat_model.invoke(msgs)
print(res.content)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A inteligência artificial (IA) é a capacidade de uma máquina ou programa de processar informações, aprender com elas e tomar decisões sem intervenção humana direta. Isso permite que as máquinas realizem tarefas que normalmente requeriam habilidades humanas, como raciocínio, aprendizado e resolução de problemas.

A IA pode ser dividida em dois tipos principais:

1. Inteligência Artificial Narrow (ANI): É a capacidade de uma máquina realizar uma tarefa específica, como reconhecer padrões em imagens ou processar linguagem natural.
2. Inteligência Artificial Geral (AGI): É a capacidade de uma máquina realizar qualquer tarefa que um ser humano pode fazer, incluindo o pensamento criativo e a tomada de decisões.

A IA é baseada em algoritmos complexos e técnicas de machine learning, que permitem que os programas aprendam com dados e melhoriem suas performances ao longo do tempo.


In [15]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Escreva um poema sobre {topic}")

prompt_template.invoke({"topic": "abacates"})

StringPromptValue(text='Escreva um poema sobre abacates')

In [16]:
prompt_template

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Escreva um poema sobre {topic}')

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente e está respondendo perguntas gerais."),
    ("user", "Explique-me em 1 parágrafo o conceito de {topic}")
])

prompt.invoke({"topic": "IA"})

ChatPromptValue(messages=[SystemMessage(content='Você é um assistente e está respondendo perguntas gerais.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explique-me em 1 parágrafo o conceito de IA', additional_kwargs={}, response_metadata={})])

In [19]:
system_prompt = "Você é um assistente e está respondendo perguntas gerais."
user_prompt = "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo 1 parágrafo."

In [21]:
user_prompt = "Explique-me em 1 parágrafo o conceito de {topic}"
user_prompt_template = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>".format(user_prompt)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente e está respondendo perguntas gerais."),
    ("user", user_prompt_template)
])

chain = prompt | llm
chain.invoke({"topic": "IA"})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'\n\nA Inteligência Artificial (IA) é a capacidade de uma máquina ou sistema processar informações, aprender com elas e tomar decisões sem intervenção humana direta. Isso ocorre quando algoritmos e programas são projetados para realizar tarefas que normalmente requerem habilidades humanas, como análise, julgamento e tomada de decisão. A IA pode ser dividida em dois tipos principais: Inteligência Artificial Nossa (Narrow AI), que é especializada em realizar uma função específica, como reconhecimento facial ou tradução de texto; e Inteligência Artificial Geral (AGI), que tem a capacidade de realizar qualquer tarefa que um humano pode fazer. A IA é baseada na ciência da computação, matemática e estatística, e seu objetivo é criar sistemas que possam aprender, adaptarem-se e melhorarem suas performances ao longo do tempo.'

In [23]:
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
{system_prompt}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [24]:
user_prompt = "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo {tamanho}."
prompt = PromptTemplate.from_template(template.format(system_prompt=system_prompt, user_prompt=user_prompt))
prompt.invoke({"topic": "IA", "tamanho": "1 frase"})

StringPromptValue(text='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de IA, de forma clara e objetiva. Escreva em no máximo 1 frase.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

challenge

In [35]:
user_prompt = "Traduza a frase {topic} para o portugês, de forma clara e objetiva e informando o idioma. Escreva em no máximo {tamanho}"
user_prompt_template = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>".format(user_prompt)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é poliglota capaz de identificar o idiomar e traduzir e está respondendo perguntas gerais."),
    ("user", user_prompt_template)
])

chain = prompt | llm
chain.invoke({"topic": "űrhajós", "tamanho": "1 frase"})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'\n\nA frase "űrhajós" é um termo húngaro que significa "astronauta", e sua tradução para o português é "astronauta". (Idioma: Húngaro)'

In [26]:
system_prompt = "Você é poliglota capaz de identificar o idiomar e traduzir e está respondendo perguntas gerais."
user_prompt = "Traduza a frase {topic} para o portugês, de forma clara e objetiva e informando o idioma. Escreva em no máximo {tamanho}."

In [27]:
prompt = PromptTemplate.from_template(template.format(system_prompt=system_prompt, user_prompt=user_prompt))
prompt.invoke({"topic": "the book is on the table", "tamanho": "1 frase"})

StringPromptValue(text='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é poliglota capaz de identificar o idiomar e traduzir e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nTraduza a frase the book is on the table para o portugês, de forma clara e objetiva e informando o idioma. Escreva em no máximo 1 frase.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [28]:
chain = prompt | llm
chain

PromptTemplate(input_variables=['tamanho', 'topic'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é poliglota capaz de identificar o idiomar e traduzir e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nTraduza a frase {topic} para o portugês, de forma clara e objetiva e informando o idioma. Escreva em no máximo {tamanho}.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')
| HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7e9cb717c450>, model_id='meta-llama/Meta-Llama-3-8B-Instruct')